In [1]:
import pandas as pd

First, we download a database which connects drugs with their uses:

In [4]:
df_uses = pd.read_csv('https://query.data.world/s/81zz5jv9bpf4n70eoaef1lvjw',index_col=0)
df_uses

URLError: <urlopen error EOF occurred in violation of protocol (_ssl.c:645)>